In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [ ]:
!pip install langchain_community

In [ ]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 21.5 MB/s eta 0:00:00


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 67.4 MB/s eta 0:00:00


## Retrieval Augmented Generation (RAG)

### Carregando Documentos - Loading

In [ ]:
# https://python.langchain.com/v0.2/docs/how_to/#document-loaders
# https://python.langchain.com/v0.2/docs/integrations/document_loaders/

import bs4
from langchain_community.document_loaders import WebBaseLoader

# Filtra o conteúdo da página por uma classe específica
bs4_strainer = bs4.SoupStrainer(class_=("mw-body-content"))

# Carrega o conteúdo da página
loader = WebBaseLoader(
    web_paths=("https://pt.wikipedia.org/wiki/Centro_de_Estudos_e_Sistemas_Avan%C3%A7ados_do_Recife",),
    bs_kwargs={"parse_only": bs4_strainer},
)

# Carrega o conteúdo da página
docs = loader.load()

len(docs[0].page_content)

6915

In [ ]:
print(docs[0].page_content[500:1000])

cife, também conhecido por seu acrônimo CESAR, é um centro de pesquisa e inovação sem fins lucrativos com sede na cidade do Recife, Pernambuco e filiais em Sorocaba, Curitiba e Manaus.[1] O CESAR foi fundado em 1996 por três professores do Centro de Informática da UFPE, Silvio Meira, Fábio Silva e Ismar Kaufman, como forma de aproximar a academia do mercado.[2]
Em 2019, o centro conta com mais de 600 funcionários e em 2018 seu faturamento foi da ordem de R$ 100 milhões.[1][3][4][5]
O CESAR é par


### Dividindo Documentos - Splitting/Chunking

In [ ]:
# https://python.langchain.com/v0.2/docs/how_to/#text-splitters

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

10

In [ ]:
print(all_splits[2].page_content)

Área Educacional[editar | editar código-fonte]
Além de atuar como centro de pesquisa e inovação, o CESAR criou um braço educacional, a CESAR School, oferecendo cursos de graduação, mestrados e doutorados profissionais.[4]
O centro iniciou sua atuação na área educacional em 2007 quando iniciou a oferta do mestrado profissional em Engenharia de Software, que foi avaliado pela CAPES em 2017 como um dos dois melhores mestrados profissionais na área de computação do país.[8] Em 2013 foi autorizada a abertura do segundo mestrado profissional, dessa vez com ênfase em Design de Artefatos Digitais.[9] A partir de 2016 o mestrado em Design passou a também ser oferecido na unidade de Manaus.[10]


### Indexando - Store

In [ ]:
# https://python.langchain.com/v0.2/docs/how_to/embed_text/

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("Onde está localizado o CESAR?")

len(retrieved_docs)

6

In [ ]:
print(retrieved_docs[0].page_content)

Área Educacional[editar | editar código-fonte]
Além de atuar como centro de pesquisa e inovação, o CESAR criou um braço educacional, a CESAR School, oferecendo cursos de graduação, mestrados e doutorados profissionais.[4]
O centro iniciou sua atuação na área educacional em 2007 quando iniciou a oferta do mestrado profissional em Engenharia de Software, que foi avaliado pela CAPES em 2017 como um dos dois melhores mestrados profissionais na área de computação do país.[8] Em 2013 foi autorizada a abertura do segundo mestrado profissional, dessa vez com ênfase em Design de Artefatos Digitais.[9] A partir de 2016 o mestrado em Design passou a também ser oferecido na unidade de Manaus.[10]


### Buscando e Recuperando Informações - Retrieve

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [ ]:
example_messages = prompt_template.invoke({
    "context": "algum contexto",
    "question": "alguma pergunta"
})

print(example_messages.to_messages())

[HumanMessage(content='Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.\n\nPergunta: alguma pergunta\n\nContexto: algum contexto\n\nResposta:\n')]


### Gerando Respostas - Generate

> Adicionar aspas



In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [ ]:
for chunk in rag_chain.stream("Onde fica o CESAR?"):
    print(chunk, end="", flush=True)

O CESAR fica em Recife, Pernambuco. Ele é um centro de pesquisa e inovação sem fins lucrativos.

## Exercícios

### Exercício 1
Faça um RAG com um pequeno arquivo de texto, contendo informações que, certamente, a LLM não conheça. Você deverá construir o arquivo e enviar para o ambiente de execução. Escolha a forma de chunking apropriada para o seu documento.

In [ ]:
texto_base = "A Liga dos Campeões da UEFA é uma competição anual de futebol em nível continental, organizada pela União das Associações Europeias de Futebol (UEFA) e disputada por clubes da Europa. É um dos torneios mais prestigiados do mundo e a competição de clubes mais prestigiada no futebol europeu, disputada pelas equipes mais bem classificadas nas respectivas ligas nacionais na temporada anterior, sendo o número de vagas atribuído consoante o coeficiente da UEFA. A final da Liga dos Campeões da UEFA é o evento esportivo anual mais visto em todo o mundo. A final da edição de 2012–13 teve o maior número de audiências até o momento, atraindo 360 milhões de telespectadores.\
              Introduzida em 1955 como Coupe des Clubs Champions Européens (em francês para Taça dos Clubes Campeões Europeus (português europeu) ou Copa dos Clubes Campeões Europeus (português brasileiro)), foi inicialmente um torneio eliminatório apenas aos campeões das ligas nacionais da Europa, com o seu vencedor considerado como o campeão europeu de clubes. A competição assumiu o nome atual em 1992, acrescentando uma fase de grupos todos contra todos em 1991 e permitindo vários participantes de determinados países desde 1997. Desde então, foi expandido e, embora a maioria das ligas nacionais da Europa ainda só possa inscrever o seu campeão, as ligas mais fortes agora oferecem até quatro equipes. Os clubes em seguida das ligas nacionais que não se qualificam para a Liga dos Campeões, são elegíveis para a competição da Liga Europa da UEFA, a segunda divisão, e, desde 2021, para a Liga Conferência Europa da UEFA, a terceira divisão.\
              No seu formato atual, a Liga dos Campeões começa em meados de julho com as quatro pré-eliminatórias. As 7 equipes apuradas entram na fase de grupos, juntando com outras 29 equipes previamente qualificadas. As 36 equipes disputam oito jogos contra 3 adversários diferentes e para efeitos de classificação são colocadas numa tabela única de classificação. As equipas classificadas entre o primeiro e o oitavo lugar apuram-se diretamente para os oitavos de final como cabeças-de-série enquanto as equipas classificadas entre o nono e o vigésimo quarto lugar disputam um play-off a duas mãos para apurar as restantes oito equipas apuradas para os oitavos de final. As equipas classificadas abaixo do vigésimo quinto lugar são eliminadas da competição. Os oitavos, quartos e meias finais são todos disputados as duas mãos. A final é disputada em partida única em terreno neutro previamente designado em maio ou início de junho.[6] O vencedor da Liga dos Campeões se qualifica para a Liga dos Campeões da temporada seguinte, bem como para a Supercopa da UEFA e para os torneios da FIFA: a Copa Intercontinental e o Mundial de Clubes.\
              Os clubes espanhóis acumulam o maior número de vitórias (19 vitórias), seguido da Inglaterra (15 vitórias) e Itália (12 vitórias). A Inglaterra tem o maior número de times vencedores, com seis clubes conquistando o título. A competição foi vencida por 23 clubes, 13 dos quais venceram mais de uma vez e oito defenderam o título com sucesso.[9] O Real Madrid é o clube com mais títulos da história da competição, tendo ganho o torneio 15 vezes, incluindo as primeiras cinco temporadas.[10] Apenas um clube venceu todas as suas partidas em um único torneio a caminho da vitória no torneio: o Bayern de Munique na temporada 2019–20.[11] O Real Madrid é o atual campeão europeu, tendo derrotado o Borussia Dortmund por 2 a 0 na final de 2024 para conquistar seu décimo quinto título, sendo o primeiro de forma invicta do clube."


from langchain_text_splitters import SpacyTextSplitter

text_splitter = SpacyTextSplitter(chunk_size=200)

texts = text_splitter.create_documents([texto_base])

len(texts)

/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


21

### Index

In [ ]:
vectorstore = FAISS.from_documents(texts, OpenAIEmbeddings())

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

retrieved_docs = retriever.invoke("Qual time mais ganhou a liga dos campeões?")

len(retrieved_docs)

3

In [ ]:
system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [ ]:
for chunk in rag_chain.stream("Qual time mais ganhou a liga dos campeões?"):
    print(chunk, end="", flush=True)

O time que mais ganhou a Liga dos Campeões é o Real Madrid, com 15 títulos.